# Sentiment Analysis of Stock Twits with Multi Layer Perceptron

### Import libs

In [1]:
#### NLP LIBS ###
import nltk
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

### Import utils function ###
from utils import get_tweets_from_db, eval_model, URI, DB_NAME
from cleaning_utils import full_preprocess

### Get Tweets from MongoDB database (`db_tweets.META`)

In [2]:
# Retrieve data from MongoDB
data_from_mongodb = get_tweets_from_db(uri=URI, db_name=DB_NAME, collection_name="AAPL")

# Create DataFrame from retrieved data
df_apple_tweets = pd.DataFrame(data_from_mongodb)

df_apple_tweets

,_id,date,content,true_sentiment
0,662691526f6491024e22dd9f,2024-04-22T16:30:04Z,$AAPL nice dip quickies $$$$$$$$$$$$$,None
1,662691536f6491024e22dda1,2024-04-22T16:27:30Z,$AAPL fucking sweatshops,bearish
2,662691536f6491024e22dda3,2024-04-22T16:26:43Z,$AAPL cnbc guy says sees another 5% down bit h...,None
3,662691546f6491024e22dda5,2024-04-22T16:26:06Z,$AAPL look for a move to the downside in the s...,bullish
4,662691556f6491024e22dda7,2024-04-22T16:24:16Z,Hey $AAPL $CMCSA &amp; $GOOG you can have my $...,None
...,...,...,...,...
137,662691d86f6491024e22deb1,2024-04-22T15:12:24Z,$AAPL 140,bearish
138,662691d86f6491024e22deb3,2024-04-22T15:12:07Z,$AAPL Y&#39;all were supposed to sell at 200. ...,bullish
139,662691e26f6491024e22deb5,2024-04-22T16:30:04Z,$AAPL nice dip quickies $$$$$$$$$$$$$,None
140,662691e46f6491024e22deb7,2024-04-22T16:27:30Z,$AAPL fucking sweatshops,bearish


## Data preprocessing

- replace the hashtags
- remove stopwords and useless punctuation

In [3]:
df_apple_tweets["cleaned_content"] = df_apple_tweets["content"].apply(full_preprocess)
df_apple_tweets

,_id,date,content,true_sentiment,cleaned_content
0,662691526f6491024e22dd9f,2024-04-22T16:30:04Z,$AAPL nice dip quickies $$$$$$$$$$$$$,None,cashtag nice dip quicki
1,662691536f6491024e22dda1,2024-04-22T16:27:30Z,$AAPL fucking sweatshops,bearish,cashtag fuck sweatshop
2,662691536f6491024e22dda3,2024-04-22T16:26:43Z,$AAPL cnbc guy says sees another 5% down bit h...,None,cashtag cnbc guy say see anoth 5 % bit ha buy ...
3,662691546f6491024e22dda5,2024-04-22T16:26:06Z,$AAPL look for a move to the downside in the s...,bullish,cashtag look move downsid short term howev exp...
4,662691556f6491024e22dda7,2024-04-22T16:24:16Z,Hey $AAPL $CMCSA &amp; $GOOG you can have my $...,None,hey cashtag cashtag cashtag cashtag cash stock
...,...,...,...,...,...
137,662691d86f6491024e22deb1,2024-04-22T15:12:24Z,$AAPL 140,bearish,cashtag 140
138,662691d86f6491024e22deb3,2024-04-22T15:12:07Z,$AAPL Y&#39;all were supposed to sell at 200. ...,bullish,cashtag suppos sell 200 buy mfkin dip earn bad...
139,662691e26f6491024e22deb5,2024-04-22T16:30:04Z,$AAPL nice dip quickies $$$$$$$$$$$$$,None,cashtag nice dip quicki
140,662691e46f6491024e22deb7,2024-04-22T16:27:30Z,$AAPL fucking sweatshops,bearish,cashtag fuck sweatshop


In [4]:
df_apple_tweets = df_apple_tweets[df_apple_tweets['true_sentiment'].isin(['bullish', 'bearish'])]

In [5]:
X = df_apple_tweets[df_apple_tweets['true_sentiment'].isin(['bullish', 'bearish'])]["cleaned_content"]

# Select features
X = df_apple_tweets["cleaned_content"]

# Select the labels
y = df_apple_tweets['true_sentiment']

### Transform the text into a digest format for the MLP (vectors)

To do so, we convert text data into numerical features using TF-IDF vectorization

In [6]:
# Convert text data into numerical features using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_vectorized = vectorizer.fit_transform(X)

## Train test split data

In [7]:
# Split the dataframe
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

### Initialize the MLP

In [8]:
mlp_classifier = MLPClassifier(random_state=42)

#### Define the hyperameters for the grid search CV

In [9]:
# Step 5: Define hyperparameters grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.001, 0.01]
}

### Apply the grid search CV

In [10]:
# Step 6: Initialize GridSearchCV
grid_search = GridSearchCV(estimator=mlp_classifier, param_grid=param_grid, cv=3, scoring='accuracy')

# Step 7: Perform GridSearchCV
grid_search.fit(X_train, y_train)

C:\Users\humic\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\humic\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\humic\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\humic\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\humic\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

GridSearchCV(cv=3, estimator=MLPClassifier(random_state=42),
             param_grid={'activation': ['logistic'], 'alpha': [0.001, 0.01],
                         'hidden_layer_sizes': [(50,), (100,), (150,)],
                         'solver': ['adam', 'sgd']},
             scoring='accuracy')

### Get the best parameters and best score

In [11]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'solver': 'adam'}
Best Score: 0.9007352941176471


### Get the predictions (forward path)

In [12]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [13]:
y_pred

array(['bullish', 'bearish', 'bearish', 'bullish', 'bullish', 'bullish',
       'bullish', 'bullish', 'bullish', 'bullish', 'bullish', 'bullish',
       'bullish'], dtype='<U7')

In [14]:
pd.DataFrame(y_pred)

,0
0,bullish
1,bearish
2,bearish
3,bullish
4,bullish
5,bullish
6,bullish
7,bullish
8,bullish
9,bullish


#### Post-Processing: Compute the predicted label
* `bullish`
* `bearish`

### Model Evaluation

To evaluate the model performance prediction we compute the confusion matrix and the classification report

In [15]:
eval_model(df_apple_tweets["true_sentiment"], df_apple_tweets["predicted_sentiment"])

KeyError: 'predicted_sentiment'

#### Store the predicted sentiment into a new MongoDB Collection

In [ ]:
## TO DO